# Octopus Agile in 2020

Some data insights into Agile this past year.

In [103]:
import altair as alt
import pandas as pd

def read_year(i):
    df = pd.read_csv(f'agile.{i}.csv', index_col=0)
    df.index = pd.to_datetime(df.index, utc=True).tz_convert('Europe/London')
    return df

df = read_year(2020)
all_years = pd.concat(
    read_year(i) for i in (2017, 2018, 2019, 2020))

## Best and worst months

As lockdown commenced, electricity prices plunged to record lows.

In [104]:
median_rate = df['rate'].resample('MS').median()
cheapest = median_rate.idxmin()
expensive = median_rate.idxmax()
print(f'The cheapest month was {cheapest.strftime("%B")} averaging {median_rate.min()} p/kWh.')
print(f'The most expensive month was {expensive.strftime("%B")} averaging {median_rate.max()} p/kWh.\n')
alt.Chart(median_rate.reset_index()).mark_bar(width=20).encode(
    alt.X('timestamp', title='Month'),
    alt.Y('rate', title='Rate / p'),
    alt.Color('rate', scale=alt.Scale(scheme='goldred'), legend=None),
).properties(width=600)

The cheapest month was May averaging 5.733 p/kWh.
The most expensive month was December averaging 11.466 p/kWh.



alt.Chart(...)

In comparison to previous years, it was a very cheap year for Agile...

In [105]:
median_rate = all_years.resample('MS').median()
median_rate['year'] = median_rate.index.year
alt.Chart(median_rate.reset_index()).mark_bar(width=10).encode(
    alt.X('timestamp', title='Month'),
    alt.Y('rate', title='Rate / p'),
    alt.Color('rate', scale=alt.Scale(scheme='goldred'), legend=None),
).properties(
    width=200
).facet(column='year').resolve_scale(
    x='independent'
)

alt.FacetChart(...)

With a record number of plunge days in 2020...

In [106]:
plunges = (all_years.rate < 0).resample('D').sum()
by_year = (plunges > 0).groupby(plunges.index.year).sum().astype(int)
alt.Chart(by_year.reset_index()).mark_bar().encode(
    alt.X('rate', title='Plunge days'),
    alt.Y('timestamp:O', title='Year'),
).properties(
    title='Plunge days by year',
    width=600,
    height=100,
)

alt.Chart(...)

In [107]:
plunges_2020 = (df.rate < 0).resample('D').sum() > 0
by_month = plunges_2020.resample('MS').sum()
alt.Chart(by_month.reset_index()).mark_bar(height=15).encode(
    alt.X('rate', title='Plunge days'),
    alt.Y('timestamp', title='Month'),
).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
).properties(
    title='Plunge days by month 2020',
    width=600,
    height=250,
)

alt.Chart(...)

With Sunday having the most plunges, closely followed by Monday. Thursday had none!

In [108]:
by_dow = plunges_2020.groupby(plunges_2020.index.day_name()).sum()
by_dow = by_dow.sort_values(ascending=False)
alt.Chart(by_dow.reset_index()).mark_bar().encode(
    alt.X('timestamp', title=None, sort=None),
    alt.Y('rate', title='Plunge days'),
).properties(
    title='Plunges by day of week',
    width=600,
    height=250,
)

alt.Chart(...)

In [109]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>